In [1]:
from langchain_openai import ChatOpenAI
from getpass import getpass
import pandas as pd
from tqdm import tqdm

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

In [2]:
df = pd.read_csv('100_crypto_messages.csv',encoding='utf-8')
df.head()

,text_id,text
0,143,Удаленная работа. От нас: Зп от 2500$
1,179,Что вы думаете о концепции децентрализованных ...
2,147,Удалённый заработок. Время 1-2часа в день. Мож...
3,157,Меня всегда интересовал вопрос безопасности хр...
4,168,"Я слышал, что некоторые банки начали экспериме..."


In [3]:
prompt_template = """Ты - профессиональный ассистент чата, который специализируется на удалении спама из чата про криптовалюты.
Твоя задача - понять является ли сообщение спамом или же это сообщение пользователя.
Если сообщение спам - верни <1>, если нет или неизвестно - <0>. Возвращаемое значение должно быть в формате int

Context: {text_input}

###

Question: Является ли данное сообщение спамом?

###

Answer: ...
"""

In [4]:
amount_list = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['text']):
    prompt = prompt_template.format(text_input=text_input) # Добавляем сообщение в промпт
    amount = llm.invoke(prompt).content # Ответ модели
    amount_list.append(amount) # Добавляем ответ в список

# перед записью ответа в датафрейм, проверьте какого типа ответы
type(amount_list[0])

100%|██████████| 100/100 [01:22<00:00,  1.22it/s]


str

In [17]:
def convert(value):
    if not(value in ['0','1']):
        return "0"
    return value
    

cl_list_1 = map(lambda x: x.replace('<',''),amount_list)
cl_list_2 = map(lambda x: x.replace('>',''),cl_list_1)
cl_list_3 = map(lambda x: convert(x),cl_list_2)
final_list = map(int,cl_list_3)
res = list(final_list)
res

df['is_spam'] = res
df.to_csv("100_crypto_messages_res.csv",index=False)